In [110]:
import pandas as pd
from dr_cr_rule import ASSET_Expense_Accounting
from dr_cr_rule import Liability_Capital_Accounting
import re

In [86]:
main_df = pd.read_excel('main_df.xlsx')
df_3 = pd.read_excel('purshase.xlsx',engine='openpyxl')

In [111]:
# df_3.columns

In [104]:
k  = {
    'J1 - Journal Number':[],
    'J2 - Date' :  [],
    'J3 - Invoice_Ref_Number' : [],
    'J4 - Ledger_Name' : [],
    'J5 - Opening Balance' :[],
    'J6 - Dr Amount' : [],
    'J7 - Cr Amount' : [],
    'J8 - Closing Balance' : [],
    'J9 - Description/Narration' : [],
    # 'J10 - Account Name': [],
}

In [105]:
journal_counter = 100
invoice_to_journal_mapping = {}
c = len(df_3)
count = 0

for i in range(c):
    j2 = df_3['Date'].iloc[i]
    j3 = df_3['Ledger Name'].iloc[i]
    j4 = df_3['Invoice Number'].iloc[i]
    
    j_5 = df_3['TOTAL'].iloc[i]
    
    invoice_ref_number = j4
    if invoice_ref_number not in invoice_to_journal_mapping:
        
        invoice_to_journal_mapping[invoice_ref_number] = journal_counter 
        k['J1 - Journal Number'].append(invoice_to_journal_mapping[invoice_ref_number])
        journal_counter += 1
    else:
        k['J1 - Journal Number'].append(invoice_to_journal_mapping[invoice_ref_number])
    
    k['J2 - Date'].append(j2)
    k['J4 - Ledger_Name'].append(j3)
    k['J3 - Invoice_Ref_Number'].append(j4)
    k['J8 - Closing Balance'].append(j_5)
    k['J9 - Description/Narration'].append(' ')

    if j3.lower() in list(main_df['C5']) :
        c1 = main_df.loc[main_df['C5'] ==  j3.lower(),'C1'].iloc[0]
        c2 = main_df.loc[main_df['C5'] ==  j3.lower(),'C2'].iloc[0]
        c3 = main_df.loc[main_df['C5'] ==  j3.lower(),'C3'].iloc[0]
        c5 = main_df.loc[main_df['C5'] ==  j3.lower(),'C5'].iloc[0]
       

    else:
        print(f"ledger name {j3 } is not present !! please enter C3 (Account Name)")

        C3 = input()
        if (main_df.loc[main_df['C3'] == C3, 'C5'] == 'NONE').any():
            main_df.loc[main_df['C3'] == C3,'C5'] =  j3.lower()
            val  = main_df.loc[main_df['C3'] == C3,'C5'].iloc[0]
            c1   = main_df.loc[main_df['C3'] == C3,'C1'].iloc[0]
            c2   = main_df.loc[main_df['C3'] == C3,'C2'].iloc[0]
            c3   = main_df.loc[main_df['C3'] == C3,'C3'].iloc[0]
    
        else:
            c1 = main_df[main_df['C3']==C3]['C1'].iloc[0]
            c2 = main_df[main_df['C3']==C3]['C2'].iloc[0]
            c3 = main_df[main_df['C3']==C3]['C3'].iloc[0]
            c4 = main_df[main_df['C3']==C3]['C4'].iloc[0]
            g_code = main_df[main_df['C3']==C3]['G_CODE'].iloc[0]

            new_data = {'C1': c1, 'C2': c2, 'C3':c3 ,'C4':c4,'C5': j3.lower() , 'G_CODE':g_code}
            main_df = main_df.append(new_data, ignore_index=True)


    if c1 == 'ASSET' or c1 == 'EXPENSE':
        if df_3['type'].iloc[i].lower() == 'dr':
            opening_balance = 0
            sys = ASSET_Expense_Accounting()
            amount = j_5
            account = c3
            cr = 0
            dr = sys.debit_transaction(amount)
            k['J6 - Dr Amount'].append(dr)
            k['J7 - Cr Amount'].append(cr)
            # k['Account Name - J10'].append(account)
            k['J5 - Opening Balance'].append(opening_balance)
    
        else:
            opening_balance = 0
            sys = ASSET_Expense_Accounting()
            amount = j_5
            account = c3
            dr = 0
            cr = sys.credit_transaction(amount)
            k['J6 - Dr Amount'].append(dr)
            k['J7 - Cr Amount'].append(cr)
            # k['Account Name - J10'].append(account)
            k['J5 - Opening Balance'].append(opening_balance)
    
        
            

In [106]:
main_df[main_df['C5']!='NONE']

,C1,C2,C3,C4,C5,G_CODE
0,ASSET,CurrentAsset,Cash,NONE,to bank,G1
3,ASSET,CurrentAsset,AccountsReceivables,NONE,laptop,G1
62,EXPENSE,Operating,Taxes,NONE,tax igst 9%,G4
79,EXPENSE,Operating,Taxes,NONE,tax cgst 9%,G4
80,ASSET,CurrentAsset,AccountsReceivables,NONE,tablet,G1
81,EXPENSE,Operating,Taxes,NONE,tax igst 25%,G4
82,EXPENSE,Operating,Taxes,NONE,tax cgst 25%,G4
83,ASSET,CurrentAsset,Cash,NONE,to cash bank,G1


In [109]:
df = pd.DataFrame(k)
df.to_excel('ledger1.xlsx')
df

,J1 - Journal Number,J2 - Date,J3 - Invoice_Ref_Number,J4 - Ledger_Name,J5 - Opening Balance,J6 - Dr Amount,J7 - Cr Amount,J8 - Closing Balance,J9 - Description/Narration
0,100,15-03-2023,AF2223-026868,Laptop,0,40000,0,40000,
1,100,15-03-2023,AF2223-026868,Tax IGST 9%,0,4000,0,4000,
2,100,15-03-2023,AF2223-026868,Tax CGST 9%,0,4000,0,4000,
3,100,15-03-2023,AF2223-026868,To Bank,0,0,48000,48000,
4,101,15-03-2023,AF2223-026869,Tablet,0,50000,0,50000,
5,101,15-03-2023,AF2223-026869,Tax IGST 25%,0,5000,0,5000,
6,101,15-03-2023,AF2223-026869,Tax CGST 25%,0,5000,0,5000,
7,101,15-03-2023,AF2223-026869,To Cash Bank,0,0,60000,60000,


In [50]:
def check_dr_cr_rule():
    dr_sum =  df['Dr Amount - J6'].sum() 
    cr_sum = df['Cr Amount - J7'].sum()
    if dr_sum != cr_sum:
        return "ledger entry fail"
    else:
        print("debit sum is ",dr_sum)
        print("credit sum is ",cr_sum)
        print("ledger entry is successful")

check_dr_cr_rule()

debit sum is  48000
credit sum is  48000
ledger entry is successful


In [62]:
df.to_excel('ledger3.xlsx')